In [93]:
# Import libraries
import numpy as np
import pandas as pd

import os

#from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso, Ridge

import string

import datetime

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [92]:
# Constants goes here
NMONTHS = 24
pd.set_option('max_columns', 200)

In [3]:
# Import data
DATA_FOLDER = '../Data/'

transactions    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv.gz'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))
test            = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv.gz'))

In [4]:
# For decrease memory consumption
def downcast_dtypes(df):
    '''
    Changes column types in the dataframe: 

            `float64` type to `float32`
            `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

In [5]:
transactions = downcast_dtypes(transactions)
items = downcast_dtypes(items)
item_categories = downcast_dtypes(item_categories)
shops = downcast_dtypes(shops)
test = downcast_dtypes(test)

In [7]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [8]:
# New function to construct base dataframes for train, validation and test
def construct_base_train_val_test(months=NMONTHS):
    """
    months - number of last months to construct base train/test matrix dataset
    This function construct grouped by months sales information for train, validation and test
    """
    # Group transactions by month, shop and item (add price mean here)
    grp_trans = transactions.groupby(['date_block_num', 'shop_id', 'item_id'], as_index=False).agg({'item_cnt_day': 'sum'})
    
    # Split dataset by months
    lst_grp_trans = []
    max_month = grp_trans.date_block_num.max() + 1
    for month in range(max_month-months-2, max_month): # Get only last year data
        month_df = grp_trans[grp_trans.date_block_num == month]
        # Rename target column and price column
        month_df.rename(index=str, columns={"item_cnt_day": "item_cnt_day_" + str(month)}, inplace=True) 
        lst_grp_trans.append(month_df)
        
        
    # Join data with test dataset
    all_data = test[['shop_id', 'item_id']].set_index(['shop_id', 'item_id'])
    for month in range(max_month-months-2, max_month): # [8..34)
        all_data = all_data.join(lst_grp_trans[month+months-max_month+2][['shop_id', 'item_id', 'item_cnt_day_'+str(month)]
                                                                        ].set_index(['shop_id', 'item_id']))
        
    #print(all_data.head())
    
    # Replace NaNs by 0
    all_data.fillna(0, inplace=True)
    
    # Get train target and validation target column
    val_target = all_data.item_cnt_day_33
    train_target = all_data.item_cnt_day_32
    
    # Calculate x_train, x_val, x_test
    x_train = all_data.copy()
    x_train.drop(['item_cnt_day_32', 'item_cnt_day_33'], axis=1, inplace=True)
    
    x_val = all_data.copy()
    first_col = str(max_month-months-2)
    x_val.drop(['item_cnt_day_' + first_col, 'item_cnt_day_33',], 
               axis=1, inplace=True)
    
    x_test = all_data.copy()
    second_col = str(max_month-months-1)
    x_test.drop(['item_cnt_day_' + first_col, 'item_cnt_day_' + second_col], axis=1, inplace=True)
    
    return x_train, x_val, x_test, train_target, val_target

In [81]:
%%time
tr_train, tr_val, tr_test, tr_target, val_target = construct_base_train_val_test()
print(tr_train.shape, tr_val.shape, tr_test.shape)

(214200, 24) (214200, 24) (214200, 24)
Wall time: 7.94 s


In [37]:
# TF-IDF transformation to the item names
tfidf = TfidfVectorizer()
item_name_matrix = tfidf.fit_transform(items.item_name)
item_name_matrix.shape

(22170, 18222)

In [48]:
%%time
tsvd = TruncatedSVD(n_components=300, random_state=26)
tsvd_item_matrix = tsvd.fit_transform(item_name_matrix)

Wall time: 10.3 s


In [51]:
# Join with the items dataframe
items = items.join(pd.DataFrame(tsvd_item_matrix, index=items.index))
items.head()

,item_name,item_id,item_category_id,0,1,2,3,4,5,6,...,290,291,292,293,294,295,296,297,298,299
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,0.000641,0.000150,6.665875e-04,-0.000096,0.000303,-0.000537,-0.000260,...,0.002997,0.001726,0.002455,-0.001184,-0.000464,0.001044,-0.002293,-0.000827,0.002190,-0.000760
1,!ABBYY FineReader 12 Professional Edition Full...,1,76,0.298384,-0.011510,-4.987161e-02,-0.098047,-0.003741,0.030858,0.004195,...,-0.024040,0.011326,-0.013554,-0.029464,0.004807,-0.002612,0.029757,0.008931,0.010555,-0.036053
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,0.000003,0.000004,-6.139519e-07,-0.000007,0.000005,0.000001,-0.000002,...,-0.000881,-0.001112,0.000067,0.000391,-0.000413,0.001365,0.000524,0.000173,-0.000476,-0.000823
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,0.000141,0.000577,9.436656e-04,-0.000011,0.000652,0.000343,0.000444,...,0.001935,0.002332,-0.002535,-0.000144,-0.002340,0.000407,-0.000345,0.001080,-0.003409,0.001682
4,***КОРОБКА (СТЕКЛО) D,4,40,0.001315,0.000337,1.050196e-03,0.000004,0.000017,0.000594,0.001523,...,0.003927,0.009936,-0.015988,0.012645,-0.002523,0.001052,0.007305,0.000453,0.005982,0.003222


In [52]:
# Add means of item_cnt_day
def add_means(train, val, test, months=NMONTHS):
    """
    train - base train dataframe
    val - base validation dataframe (train, shifted by 1 month further)
    test - base test dataframe (val, shifted by 1 month further)
    This function add sliding means by last n column values (last n months)
    """
    # Define filters for price and item count features separation
    train_filter_item_cnt_col = [col for col in train if col.startswith('item_cnt_day')]
    val_filter_item_cnt_col = [col for col in val if col.startswith('item_cnt_day')]
    test_filter_item_cnt_col = [col for col in test if col.startswith('item_cnt_day')]
    
    # Get 6 separate dataframes
    train_item_cnt_df = train[train_filter_item_cnt_col]
    val_item_cnt_df = val[val_filter_item_cnt_col]
    test_item_cnt_df = test[test_filter_item_cnt_col]
    
    # Add features to train
    np_train_count = np.array(train_item_cnt_df)
    
    for i in range(1, months+1):
        train['mean_cnt_' + str(i)] = pd.Series(np_train_count[:, -i:].mean(axis=1), index=train.index)
        
    # Add features to validation
    np_val_count = np.array(val_item_cnt_df)
    
    for i in range(1, months+1):
        val['mean_cnt_' + str(i)] = pd.Series(np_val_count[:, -i:].mean(axis=1), index=train.index)
        
    # Add features to test
    np_test_count = np.array(test_item_cnt_df)
    
    for i in range(1, months+1):
        test['mean_cnt_' + str(i)] = pd.Series(np_test_count[:, -i:].mean(axis=1), index=test.index)
        
    return train, val, test

In [82]:
%%time
tr_train, tr_val, tr_test = add_means(tr_train, tr_val, tr_test)
print(tr_train.shape, tr_val.shape, tr_test.shape)

(214200, 48) (214200, 48) (214200, 48)
Wall time: 256 ms


In [60]:
# Add some mean encoded features
def get_month_mean_encodings(train, val, test):
    """
    train - base train dataframe
    val - base validation dataframe (train, shifted by 1 month further)
    test - base test dataframe (val, shifted by 1 month further)
    Function calculate cumsum divided by cumcount for current shop and current item saled quantities,
    also it makes the same transformations for current month price
    We aggregate info by months
    """
    # Drop indexes
    train.reset_index(inplace=True)
    val.reset_index(inplace=True)
    test.reset_index(inplace=True)
    
    # Get dataframes with sold item quantities, grouped by shop and item
    
    # Define item count column filters
    train_filter_cnt_col = [col for col in train if col.startswith('item_cnt_day')]
    val_filter_cnt_col = [col for col in val if col.startswith('item_cnt_day')]
    test_filter_cnt_col = [col for col in test if col.startswith('item_cnt_day')]
    
    # Aggregate train by shop and item
    train_shop_agg_cnt_df = train.groupby(['shop_id'])[train_filter_cnt_col].agg(['sum'])
    train_item_agg_cnt_df = train.groupby(['item_id'])[train_filter_cnt_col].agg(['sum'])
    
    # Aggregate validation set by shop and item
    val_shop_agg_cnt_df = val.groupby(['shop_id'])[val_filter_cnt_col].agg(['sum'])
    val_item_agg_cnt_df = val.groupby(['item_id'])[val_filter_cnt_col].agg(['sum'])
    
    # Aggregate test set by shop and item
    test_shop_agg_cnt_df = test.groupby(['shop_id'])[test_filter_cnt_col].agg(['sum'])
    test_item_agg_cnt_df = test.groupby(['item_id'])[test_filter_cnt_col].agg(['sum'])
    

    # Now we get train cumulative sum of this features on rows
    train_shop_agg_cnt_df = train_shop_agg_cnt_df.cumsum(axis=1)
    train_item_agg_cnt_df = train_item_agg_cnt_df.cumsum(axis=1)
    
    # Get validation cumulative sum of this features on rows
    val_shop_agg_cnt_df = val_shop_agg_cnt_df.cumsum(axis=1)
    val_item_agg_cnt_df = val_item_agg_cnt_df.cumsum(axis=1)

    # Get test cumulative sum of this features on rows
    test_shop_agg_cnt_df = test_shop_agg_cnt_df.cumsum(axis=1)
    test_item_agg_cnt_df = test_item_agg_cnt_df.cumsum(axis=1)

    # Calculate arithmetic progression (1, 2, 3, 4, ...) with months length
    np_to_divide = np.array(train_item_agg_cnt_df.columns.labels[0] + 1, dtype=float)
    
    # Divide train cumulative sum by column number (np.cumcount emulation)
    train_shop_agg_cnt_df = train_shop_agg_cnt_df / np_to_divide
    train_item_agg_cnt_df = train_item_agg_cnt_df / np_to_divide
    
    # Divide validation cumulative sum by column number (np.cumcount emulation)
    val_shop_agg_cnt_df = val_shop_agg_cnt_df / np_to_divide
    val_item_agg_cnt_df = val_item_agg_cnt_df / np_to_divide
    
    # Divide test cumulative sum by column number (np.cumcount emulation)
    test_shop_agg_cnt_df = test_shop_agg_cnt_df / np_to_divide
    test_item_agg_cnt_df = test_item_agg_cnt_df / np_to_divide
    
    
    # Move index to columns of train daraframes
    train_shop_agg_cnt_df.reset_index(inplace=True)
    train_item_agg_cnt_df.reset_index(inplace=True)
    
    # Move index to columns of validation daraframes
    val_shop_agg_cnt_df.reset_index(inplace=True)
    val_item_agg_cnt_df.reset_index(inplace=True)
    
    # Move index to columns of test daraframes
    test_shop_agg_cnt_df.reset_index(inplace=True)
    test_item_agg_cnt_df.reset_index(inplace=True)

    
    # Join all train mean encoding features with initial data
    train_ext = pd.merge(train, train_shop_agg_cnt_df, how='left', left_on='shop_id', right_on='shop_id')
    train_ext = pd.merge(train_ext, train_item_agg_cnt_df, how='left', left_on='item_id', right_on='item_id')

    # Join all validation mean encoding features with initial data
    val_ext = pd.merge(val, val_shop_agg_cnt_df, how='left', left_on='shop_id', right_on='shop_id')
    val_ext = pd.merge(val_ext, val_item_agg_cnt_df, how='left', left_on='item_id', right_on='item_id')
                       
    # Join all test mean encoding features with initial data
    test_ext = pd.merge(test, test_shop_agg_cnt_df, how='left', left_on='shop_id', right_on='shop_id')
    test_ext = pd.merge(test_ext, test_item_agg_cnt_df, how='left', left_on='item_id', right_on='item_id')

    return train_ext, val_ext, test_ext

In [83]:
%%time
tr_train, tr_val, tr_test = get_month_mean_encodings(tr_train, tr_val, tr_test)

# Drop duplicate column
tr_train.drop(['item_id_x'], axis=1, inplace=True)
tr_val.drop(['item_id_x'], axis=1, inplace=True)
tr_test.drop(['item_id_x'], axis=1, inplace=True)

print(tr_train.shape, tr_val.shape, tr_test.shape)

(214200, 98) (214200, 98) (214200, 98)
Wall time: 2.42 s


In [70]:
tr_train.head()

,item_id,shop_id,item_cnt_day_8_x,item_cnt_day_9_x,item_cnt_day_10_x,item_cnt_day_11_x,item_cnt_day_12_x,item_cnt_day_13_x,item_cnt_day_14_x,item_cnt_day_15_x,...,"(item_cnt_day_22_y, sum)","(item_cnt_day_23_y, sum)","(item_cnt_day_24_y, sum)","(item_cnt_day_25_y, sum)","(item_cnt_day_26_y, sum)","(item_cnt_day_27_y, sum)","(item_cnt_day_28_y, sum)","(item_cnt_day_29_y, sum)","(item_cnt_day_30_y, sum)","(item_cnt_day_31_y, sum)"
0,5037,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,19.133333,30.4375,33.529412,33.444444,33.157895,32.75,35.095238,37.818182,38.478261,41.750000
1,5320,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
2,5233,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.40,3.571429,8.318182,9.434783,15.250000
3,5232,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,2.666667
4,5268,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000


In [84]:
# Add tf-idf item name features
tr_train = tr_train.join(items, how='left', on='item_id', rsuffix='_itm')
tr_val = tr_val.join(items, how='left', on='item_id', rsuffix='_itm')
tr_test = tr_test.join(items, how='left', on='item_id', rsuffix='_itm')

# Drop useless columns
tr_train.drop(['item_name', 'item_id_itm'], axis=1, inplace=True)
tr_val.drop(['item_name', 'item_id_itm'], axis=1, inplace=True)
tr_test.drop(['item_name', 'item_id_itm'], axis=1, inplace=True)

print(tr_train.shape, tr_val.shape, tr_test.shape)

(214200, 399) (214200, 399) (214200, 399)


In [88]:
tr_train.iloc[:, -305:].head()

,"(item_cnt_day_28_y, sum)","(item_cnt_day_29_y, sum)","(item_cnt_day_30_y, sum)","(item_cnt_day_31_y, sum)",item_category_id,0,1,2,3,4,...,290,291,292,293,294,295,296,297,298,299
0,35.095238,37.818182,38.478261,41.750000,19,0.079082,0.002221,0.020591,0.190210,0.019453,...,0.017789,-0.018557,0.000573,0.023761,0.032483,0.004814,-0.028143,0.054562,-0.055948,0.010696
1,0.000000,0.000000,0.000000,0.000000,55,0.035927,0.001655,0.005497,0.055388,0.003611,...,-0.008180,0.012084,0.007222,-0.014114,0.008591,-0.013691,0.015844,-0.005519,0.006485,-0.006548
2,3.571429,8.318182,9.434783,15.250000,19,0.238340,-0.005698,-0.035638,0.227311,0.021017,...,0.000413,0.012644,-0.028473,-0.030405,-0.007984,0.018558,-0.002854,-0.001155,0.014952,-0.011517
3,0.000000,0.000000,0.000000,2.666667,23,0.247804,-0.006887,-0.041150,0.322485,0.031554,...,0.012888,0.012114,-0.005169,-0.017907,-0.002717,0.014629,0.001293,-0.013155,-0.006749,-0.023866
4,0.000000,0.000000,0.000000,0.000000,20,0.248577,-0.007308,-0.034032,0.181913,0.016260,...,-0.002899,-0.002566,-0.000134,-0.004104,-0.020138,0.003054,0.007063,0.003258,0.008193,-0.004893


In [89]:
# Extract city column from shops dataframe
shop_name_split = [el.split(' ') for el in shops.shop_name.values]
shops['city'] = pd.Series([el[0] for el in shop_name_split], index=shops.index)
shops.head(10)

,shop_name,shop_id,city
0,"!Якутск Орджоникидзе, 56 фран",0,!Якутск
1,"!Якутск ТЦ ""Центральный"" фран",1,!Якутск
2,"Адыгея ТЦ ""Мега""",2,Адыгея
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха
4,"Волжский ТЦ ""Волга Молл""",4,Волжский
5,"Вологда ТРЦ ""Мармелад""",5,Вологда
6,"Воронеж (Плехановская, 13)",6,Воронеж
7,"Воронеж ТРЦ ""Максимир""",7,Воронеж
8,"Воронеж ТРЦ Сити-Парк ""Град""",8,Воронеж
9,Выездная Торговля,9,Выездная


In [90]:
# Add city feature
tr_train = pd.merge(tr_train, shops, how='left', left_on='shop_id', right_on='shop_id')
tr_val = pd.merge(tr_val, shops, how='left', left_on='shop_id', right_on='shop_id')
tr_test = pd.merge(tr_test, shops, how='left', left_on='shop_id', right_on='shop_id')
print(tr_train.shape, tr_val.shape, tr_test.shape)

# Drop unnecessary shop_name column
tr_train.drop(['shop_name'], axis=1, inplace=True)
tr_val.drop(['shop_name'], axis=1, inplace=True)
tr_test.drop(['shop_name'], axis=1, inplace=True)
print(tr_train.shape, tr_val.shape, tr_test.shape)

(214200, 401) (214200, 401) (214200, 401)
(214200, 400) (214200, 400) (214200, 400)


In [91]:
# Factorize city name
tr_train['city'] = tr_train['city'].factorize()[0]
tr_val['city'] = tr_val['city'].factorize()[0]
tr_test['city'] = tr_test['city'].factorize()[0]

In [ ]:
# Prepare and scale data
np_train = np.array(tr_train)
np_val = np.array(tr_val)
np_test = np.array(tr_test)

tr_target_clip = np.clip(np.array(tr_target), 0, 20)
val_target_clip = np.clip(np.array(val_target), 0, 20)

In [97]:
tr_train.describe()

,item_id,shop_id,item_cnt_day_8_x,item_cnt_day_9_x,item_cnt_day_10_x,item_cnt_day_11_x,item_cnt_day_12_x,item_cnt_day_13_x,item_cnt_day_14_x,item_cnt_day_15_x,item_cnt_day_16_x,item_cnt_day_17_x,item_cnt_day_18_x,item_cnt_day_19_x,item_cnt_day_20_x,item_cnt_day_21_x,item_cnt_day_22_x,item_cnt_day_23_x,item_cnt_day_24_x,item_cnt_day_25_x,item_cnt_day_26_x,item_cnt_day_27_x,item_cnt_day_28_x,item_cnt_day_29_x,item_cnt_day_30_x,item_cnt_day_31_x,mean_cnt_1,mean_cnt_2,mean_cnt_3,mean_cnt_4,mean_cnt_5,mean_cnt_6,mean_cnt_7,mean_cnt_8,mean_cnt_9,mean_cnt_10,mean_cnt_11,mean_cnt_12,mean_cnt_13,mean_cnt_14,mean_cnt_15,mean_cnt_16,mean_cnt_17,mean_cnt_18,mean_cnt_19,mean_cnt_20,mean_cnt_21,mean_cnt_22,mean_cnt_23,mean_cnt_24,"('item_cnt_day_8', 'sum')_x","('item_cnt_day_9', 'sum')_x","('item_cnt_day_10', 'sum')_x","('item_cnt_day_11', 'sum')_x","('item_cnt_day_12', 'sum')_x","('item_cnt_day_13', 'sum')_x","('item_cnt_day_14', 'sum')_x","('item_cnt_day_15', 'sum')_x","('item_cnt_day_16', 'sum')_x","('item_cnt_day_17', 'sum')_x","('item_cnt_day_18', 'sum')_x","('item_cnt_day_19', 'sum')_x","('item_cnt_day_20', 'sum')_x","('item_cnt_day_21', 'sum')_x","('item_cnt_day_22', 'sum')_x","('item_cnt_day_23', 'sum')_x","('item_cnt_day_24', 'sum')_x","('item_cnt_day_25', 'sum')_x","('item_cnt_day_26', 'sum')_x","('item_cnt_day_27', 'sum')_x","('item_cnt_day_28', 'sum')_x","('item_cnt_day_29', 'sum')_x","('item_cnt_day_30', 'sum')_x","('item_cnt_day_31', 'sum')_x","(item_cnt_day_8_y, sum)","(item_cnt_day_9_y, sum)","(item_cnt_day_10_y, sum)","(item_cnt_day_11_y, sum)","(item_cnt_day_12_y, sum)","(item_cnt_day_13_y, sum)","(item_cnt_day_14_y, sum)","(item_cnt_day_15_y, sum)","(item_cnt_day_16_y, sum)","(item_cnt_day_17_y, sum)","(item_cnt_day_18_y, sum)","(item_cnt_day_19_y, sum)","(item_cnt_day_20_y, sum)","(item_cnt_day_21_y, sum)","(item_cnt_day_22_y, sum)","(item_cnt_day_23_y, sum)","(item_cnt_day_24_y, sum)","(item_cnt_day_25_y, sum)","(item_cnt_day_26_y, sum)","(item_cnt_day_27_y, sum)","(item_cnt_day_28_y, sum)","(item_cnt_day_29_y, sum)","(item_cnt_day_30_y, sum)","(item_cnt_day_31_y, sum)",item_category_id,0,...,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,city
count,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,214200.000000,2.142000e+05,...,214200.000000,2.142000e+05,214200.000000,214200.00000

In [ ]:
# To logarithm: item_id, ('item_cnt_day_8', 'sum')_x - ('item_cnt_day_31', 'sum')_x,
# (item_cnt_day_8_y, sum) - (item_cnt_day_31_y, sum), city